<a href="https://colab.research.google.com/github/akrysm/rnaseq/blob/main/RNA_seq_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pythonでの準備: ランタイム設定で「Python 3」にする

In [ ]:
#@title Google Driveをマウント
# ランタイムタイプをPythonに変更
from google.colab import drive
drive.mount('/content/drive')

# ランタイムタイプをRに変更

Mounted at /content/drive


#Rでの準備: ランタイム設定で「R」にする

In [ ]:
#@title インストール&ロードパッケージ, 関数定義
#インストールパッケージ
install.packages("BiocManager")
install.packages("data.tree")
install.packages("fs")
install.packages("openxlsx")

BiocManager::install("edgeR")

#ロードパッケージ
library(edgeR)
library(fs)
library(data.tree)
library(lubridate)
library(magrittr)
library(openxlsx)
library(readxl)
library(stringr)
library(tidyverse)

# 関数
# 再帰的にフォルダ内のファイルとサブフォルダーを表示する関数
list_files_recursive <- function(path) {
  dir_contents <- list.files(path, full.names = TRUE)
  for (item in dir_contents) {
    if (file.info(item)$isdir) {
      cat("フォルダ:", item, "\n")
      list_files_recursive(item)
    } else {
      cat("ファイル:", item, "\n")
    }
  }
}

# フォルダーのツリー構造を表示する関数
display_tree <- function(path) {
  system(paste("tree", path))
}

# フォルダが存在しない場合は作成する関数
create_folder <- function(folder_path) {
  if (!file.exists(folder_path)) {
    dir.create(folder_path, recursive = TRUE)
    print(paste("Folder created:", folder_path))
  } else {
    print(paste("Folder already exists:", folder_path))
  }
}

# 各組織におけるPV_SV, PC10_PV, PC30_PVの比較結果をエクセルに抽出(P < 0.05, FDR < 0.05)
extract_gene_list <- function(read_data, tissues, sample_ids, P_VALUE = 0.05, FDR_VALUE = 0.05) {
        for (tissue in tissues) {
                # サンプルIDを用いて比較のリストを作成
                comparisons <- c()
                for (i in 1:length(sample_ids)) {
                        for (j in (i+1):length(sample_ids)) {
                                comparisons <- c(comparisons, paste0(tissue, "_", sample_ids[i], "-", tissue, "_", sample_ids[j]))
                        }
                }

                # 指定した組織に対応するデータを抽出
                data_tissue <- read_data %>% filter(Comparison %in% comparisons)

                # P値とFDRでフィルタリング
                data_tissue_p <- data_tissue %>% filter(PValue < P_VALUE)
                data_tissue_fdr <- data_tissue %>% filter(FDR < FDR_VALUE)

                # 各条件でのフィルタリング
                data_tissue_fdr_PV_SV_up <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PV-", tissue, "_SV"), Regulation == "up")
                data_tissue_fdr_PV_SV_down <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PV-", tissue, "_SV"), Regulation == "down")
                data_tissue_fdr_PV_SV_both <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PV-", tissue, "_SV"), Regulation %in% c("up", "down"))

                data_tissue_fdr_PC10_PV_up <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PC10-", tissue, "_PV"), Regulation == "up")
                data_tissue_fdr_PC10_PV_down <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PC10-", tissue, "_PV"), Regulation == "down")
                data_tissue_fdr_PC10_PV_both <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PC10-", tissue, "_PV"), Regulation %in% c("up", "down"))

                data_tissue_fdr_PC30_PV_up <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PC30-", tissue, "_PV"), Regulation == "up")
                data_tissue_fdr_PC30_PV_down <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PC30-", tissue, "_PV"), Regulation == "down")
                data_tissue_fdr_PC30_PV_both <- data_tissue_fdr %>% filter(Comparison == paste0(tissue, "_PC30-", tissue, "_PV"), Regulation %in% c("up", "down"))

                # ファイル名を指定
                file_name <- paste0(tissue, "_gene_list.xlsx")

                # 新しいExcelブックを作成
                wb <- createWorkbook()

                # シートを追加
                addWorksheet(wb, "P")
                addWorksheet(wb, "FDR")
                addWorksheet(wb, "FDR PV-SV Up")
                addWorksheet(wb, "FDR PV-SV Down")
                addWorksheet(wb, "FDR PV-SV Both")
                addWorksheet(wb, "FDR PC10-PV Up")
                addWorksheet(wb, "FDR PC10-PV Down")
                addWorksheet(wb, "FDR PC10-PV Both")
                addWorksheet(wb, "FDR PC30-PV Up")
                addWorksheet(wb, "FDR PC30-PV Down")
                addWorksheet(wb, "FDR PC30-PV Both")

                # データをシートに書き込む
                writeData(wb, sheet = "P", data_tissue_p)
                writeData(wb, sheet = "FDR", data_tissue_fdr)
                writeData(wb, sheet = "FDR PV-SV Up", data_tissue_fdr_PV_SV_up)
                writeData(wb, sheet = "FDR PV-SV Down", data_tissue_fdr_PV_SV_down)
                writeData(wb, sheet = "FDR PV-SV Both", data_tissue_fdr_PV_SV_both)
                writeData(wb, sheet = "FDR PC10-PV Up", data_tissue_fdr_PC10_PV_up)
                writeData(wb, sheet = "FDR PC10-PV Down", data_tissue_fdr_PC10_PV_down)
                writeData(wb, sheet = "FDR PC10-PV Both", data_tissue_fdr_PC10_PV_both)
                writeData(wb, sheet = "FDR PC30-PV Up", data_tissue_fdr_PC30_PV_up)
                writeData(wb, sheet = "FDR PC30-PV Down", data_tissue_fdr_PC30_PV_down)
                writeData(wb, sheet = "FDR PC30-PV Both", data_tissue_fdr_PC30_PV_both)

                # Excelファイルを保存
                saveWorkbook(wb, file_name, overwrite = TRUE)
        }
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘Rcpp’


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.18 (BiocManager 1.30.22), R 4.3.3 (2024-02-29)

Installing package(s) 'BiocVersion', 'edgeR'

also installing the dependencies ‘statmod’, ‘limma’, ‘locfit’


Old packages: 'bit', 'bslib', 'callr', 'curl', 'data.table', 'DBI', 'dbplyr',
  'devtools', 'digest', 'gargle', 'ggplot2', 'highr', 'htmltools', 'isoband',
  'openssl', 'pkgbuild', 'processx', 'ragg', 'remotes', 'rmarkdown',
  'rstudioapi', 'rvest', 'sas

In [ ]:
#@title  フォルダ作成
# 新規フォルダのパス
DIR_NEW_FOLDER <- "/content/drive/MyDrive/RNAseq"  #@param ["/content/drive/MyDrive/RNAseq", "option1", "option2"] {allow-input: true}
NEW_FOLDER_NAME <- "R"  #@param ["R", "option1", "option2"] {allow-input: true}

# フォルダが存在しない場合は作成する
create_folder(str_c(DIR_NEW_FOLDER, "/", NEW_FOLDER_NAME, sep = ""))

[1] "Folder already exists: /content/drive/MyDrive/RNAseq/R"


In [ ]:
#@title  (コンテンツ表示)
# Google Driveがマウントされたパスを表示
# drive_path <- '/content/drive/MyDrive/RNAseq' #@param ["/content", "/content/drive/MyDrive/RNAseq", "/content/drive/MyDrive/RNAseq/rawdata", "option"] {allow-input: true}
# cat("Google Driveのパス:", drive_path, "\n")

# Google Drive内のファイルやフォルダーを表示
# dir_contents <- list.files(drive_path)
# cat("Google Driveの内容:\n")
# cat(dir_contents, sep = "\n")

# Google Drive内のファイルやフォルダーを再帰的に表示
# cat("Google Driveの中身:\n")
# list_files_recursive(drive_path)

# 再帰的にフォルダーを表示
# recursive_list <- dir_ls(drive_path, recurse = TRUE)
# print(recursive_list)

# Rootの表示
# rlang::env_parents(environment()) %>% purrr::map(environmentName)  %>%
#   setNames(., nm = stringr::str_remove(., "package:")) %>%
#   purrr::map(function(.) setNames(as.list(x=.), str_remove(., "package:") )) %>% rev %>%
#   data.tree::as.Node()

# データ整形

In [ ]:
# ディレクトリ・パス設定
# ワーキングディレクトリとファイルパス
DIR_R <- "/content/drive/MyDrive/RNAseq/R"  #@param ["/content/drive/MyDrive/RNAseq/R", "option1", "option2"] {allow-input: true}
DIR_WORK = "/content/drive/MyDrive/RNAseq/R"  #@param ["/content/drive/MyDrive/RNAseq/featureCounts", "/content/drive/MyDrive/RNAseq/R", "option1"] {allow-input: true}

# ワーキングディレクトリへ移動
setwd(DIR_WORK)
print(paste("現在のワーキングディレクトリ:", getwd()))


[1] "現在のワーキングディレクトリ: /content/drive/MyDrive/RNAseq/R"


In [ ]:
# データのコピー
# ファイルをコピーする元と先のパス
source_path <- "/content/drive/MyDrive/RNAseq/featureCounts/featureCounts.txt"  #@param ["/content/drive/MyDrive/RNAseq/featureCounts/featureCounts.txt", "/content/drive/MyDrive/共有/名城大学農学部共有フォルダ/兒島孝明先生/RNA-seq/20240117_PCPxCLZ_PFC_NAc_HIP/吉見先生/240112_featureCounts/240112_featurecounts.txt", "option1"] {allow-input: true}
target_path <- "/content/drive/MyDrive/RNAseq/R/featureCounts.txt"  #@param ["/content/drive/MyDrive/RNAseq/R/featureCounts.txt", "option1", "option2"] {allow-input: true}

# ファイルをコピー
file.copy(from = source_path, to = target_path, overwrite = FALSE)


[1] TRUE

In [ ]:
# データの書き出し(2-6列目削除)
# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/R"  #@param ["/content/drive/MyDrive/RNAseq/R", "option1", "option2"] {allow-input: true}
file_path <- "featureCounts.txt"  #@param ["featureCounts.txt", "/content/drive/MyDrive/RNAseq/R/featureCounts.txt", "option1"] {allow-input: true}
new_file_path <- "featureCounts_rm2to6.txt"  #@param ["featureCounts_rm2to6.txt", "/content/drive/MyDrive/RNAseq/R/featureCounts_rm2to6.txt", "option1"] {allow-input: true}

# ワーキングディレクトリへ移動
setwd(DIR_WORK)
print(paste("現在のワーキングディレクトリ:", getwd()))

# ファイルを読み込む
read_data <- read.table(file_path, header = TRUE, skip = 1, sep = "\t")
print("処理前")
head(read_data)
read_data <- read_data %>% dplyr::select(-c(2:6))
print("処理後")
head(read_data)
# colnames(read_data) %>% length()
colnames(read_data) <- c("Geneid", "HIP_PC10", "HIP_PC30", "HIP_PV", "HIP_SC10", "HIP_SC30", "HIP_SV", "NAc_PC10", "NAc_PC30", "NAc_PV", "NAc_SC10", "NAc_SC30", "NAc_SV", "PFC_PC10", "PFC_PC30", "PFC_PV", "PFC_SC10", "PFC_SC30", "PFC_SV")

# 新しいファイルに書き出す
write.table(read_data, file = new_file_path, sep = "\t", quote = FALSE, row.names = FALSE)

[1] "現在のワーキングディレクトリ: /content/drive/MyDrive/RNAseq/R"
[1] "処理前"


,Geneid,Chr,Start,End,Strand,Length,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_PC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_PC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_PV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_SC10_sorted.bam,⋯,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_PV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_SC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_SC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_SV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_PC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_PC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_PV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_SC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_SC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_SV_sorted.bam
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Xkr4,NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7,3286248;3286248;3393973;3422511;3491925;3491925;3491925;3491925;3740775;3740775;3740775;3740775,3287191;3287191;3393983;3422536;3492124;3492124;3492124;3492124;3741571;3741571;3741571;3741571,-;-;-;-;-;-;-;-;-;-;-;-,1978,297,294,179,184,⋯,94,130,147,88,184,121,184,172,225,194
2,Rp1,NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7,4186161;4189891;4190230;4190238;4212835;4212835;4218035;4218035;4218835;4218835;4234078;4234078;4240428;4240428;4267757;4267757;4276883;4276883;4296834;4296834;4298666;4298666;4301276;4301276;4313356;4313356;4313640;4313640;4313766;4313766;4315254;4315254;4331750;4331750;4337692;4337692;4354989;4354989;4363149;4363149;4363207;4381493;4381493;4414826;4422133;4422133;4422133;4422133;4422425;4422425;4422425;4422425;4479393;4479393,4186171;4189935;4190296;4190296;4212989;4212989;4218186;4218186;4218967;4218967;4234164;4234164;4240627;4240627;4267864;4267864;4277060;4277060;4297046;4297046;4298842;4298842;4301367;4301367;4313485;4313485;4313671;4313671;4313842;4313842;4315329;4315329;4331828;4331828;4337843;4337843;4355121;4355121;4363235;4363235;4363235;4381656;4381656;4420314;4422304;4422304;4422304;4422304;4423060;4423048;4423048;4423060;4479410;4479410,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-,8863,4,7,2,6,⋯,8,2,0,16,0,6,0,6,2,0
3,Sox17,NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7,4561942;4561942;4561942;4561942;4561942;4561942;4561942;4561942;4561942;4561942;4563323;4563323;4563323;4563323;4563323;4563323;4563995;4563995;4565359;4565359;4565359,4562891;4562891;4562891;4562891;4562891;4562891;4562891;4562814;4562891;4562891;4563629;4563629;4563629;4563629;4563629;4563629;4564086;4564086;4565368;4565378;4565378,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-,1369,120,138,81,113,⋯,129,184,159,142,176,119,115,117,163,145
4,Mrpl15,NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7;NC_000067.7,4844678;4846690;4847748;4847748;4852447;48

[1] "処理後"


,Geneid,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_PC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_PC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_PV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_SC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_SC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.HIP_SV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_PC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_PC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_PV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_SC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_SC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.NAc_SV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_PC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_PC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_PV_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_SC10_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_SC30_sorted.bam,X.content.drive.MyDrive.RNAseq.bamfiles.PFC_SV_sorted.bam
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Xkr4,297,294,179,184,229,230,173,75,94,130,147,88,184,121,184,172,225,194
2,Rp1,4,7,2,6,4,6,8,2,8,2,0,16,0,6,0,6,2,0
3,Sox17,120,138,81,113,108,102,129,137,129,184,159,142,176,119,115,117,163,145
4,Mrpl15,1347,1427,976,1252,1076,1167,1207,950,1026,1415,1062,986,1129,923,1306,1344,1183,982
5,Lypla1,436,541,393,438,470,510,325,319,265,306,309,282,483,334,516,586,490,489
6,Tcea1,1557,1630,1245,1194,1384,1392,1185,1058,1243,1448,1069,1320,1390,1068,1541,1609,1446,1161


#データ解析: edgeR

In [ ]:
# インストールパッケージ
# install.packages("BiocManager")
# BiocManager::install("edgeR")

# ロードパッケージ
# library(edgeR)
# library(lubridate)

In [ ]:
# データの再読み込み
# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/R"  #@param ["/content/drive/MyDrive/RNAseq/R", "option1", "option2"] {allow-input: true}
file_path <- "featureCounts_rm2to6.txt"  #@param ["featureCounts_rm2to6.txt", "/content/drive/MyDrive/RNAseq/R/featureCounts_rm2to6.txt", "option2"] {allow-input: true}
new_file_path <- "featureCounts_rm2to6_edger.txt"  #@param ["featureCounts_rm2to6_edger.txt", "/content/drive/MyDrive/RNAseq/R/featureCounts_rm2to6_edger", "option1"] {allow-input: true}

# ワーキングディレクトリへ移動
setwd(DIR_WORK)
print(paste("現在のワーキングディレクトリ:", getwd()))

# ファイルを読み込む
read_data <- read.table(file_path , stringsAsFactors=F, header=T, row.names=1)
# str(read_data)
# head(read_data)
group <- factor(c("HIP_PC10", "HIP_PC30", "HIP_PV", "HIP_SC10", "HIP_SC30", "HIP_SV", "NAc_PC10", "NAc_PC30", "NAc_PV", "NAc_SC10", "NAc_SC30", "NAc_SV", "PFC_PC10", "PFC_PC30", "PFC_PV", "PFC_SC10", "PFC_SC30", "PFC_SV"))
y <- DGEList(counts=read_data,group=group)

[1] "現在のワーキングディレクトリ: /content/drive/MyDrive/RNAseq/R"


In [ ]:
# 低発現遺伝子のフィルタリング
keep <- filterByExpr(y, group=group)
y <- y[keep, , keep.lib.sizes=FALSE]
y <- calcNormFactors(y)    #TMM正規化
# y$samples

In [ ]:
#@title exactTest

# 処理開始時刻を記録
start_time <- Sys.time()
start_time_jst <- with_tz(start_time, tzone = "Asia/Tokyo")

# 1連なのでBCV (square-rootdispersion)を0.1とする
bcv <- 0.1

# 結果ファイルの名前を設定
output_file <- "edgeR_result_bcv_0.1.txt"  #@param ["edgeR_result_bcv_0.1.txt", "/content/drive/MyDrive/RNAseq/R/edgeR_result_bcv_0.1.txt", "option2"] {allow-input: true}

# グループペアの組み合わせを取得
group_combinations <- combn(levels(group), 2, simplify = FALSE)

# 各グループペアに対してexactTestを実行し、結果をデータフレームに連結する
# 全結果データフレームの初期化
all_results_df <- data.frame()

for (i in seq_along(group_combinations)) {
 pair <- group_combinations[[i]]
 y_pair <- y[, group %in% pair]
 et_result <- exactTest(y_pair, dispersion=bcv^2)

 # topTagsを使用して結果を取得
 top_result <- topTags(et_result, n=Inf)

 # 結果をデータフレームに変換
 pair_results_df <- as.data.frame(top_result$table)
 pair_results_df$Gene <- rownames(pair_results_df)
 pair_results_df$Comparison <- paste(pair, collapse = "-")

 # FDR計算
 pair_results_df$FDR <- p.adjust(pair_results_df$PValue, method = "BH")

 # 全結果データフレームに追加
 all_results_df <- rbind(all_results_df, pair_results_df)
}

# 列名を整理（Gene列を前に、Comparison列を後ろに移動）
all_results_df <- all_results_df[, c("Comparison", "Gene", "logFC", "logCPM", "PValue", "FDR")]

# ファイルに全結果を書き込み
write.table(all_results_df, file=output_file, sep="\t", quote=FALSE, row.names=FALSE)

# 処理終了時刻を記録
end_time <- Sys.time()
end_time_jst <- with_tz(end_time, tzone = "Asia/Tokyo")

# 処理にかかった時間を計算
processing_time <- end_time - start_time

# 処理開始時刻と終了時刻を表示
print(paste("処理開始時刻（日本時間）:", start_time_jst))
print(paste("処理終了時刻（日本時間）:", end_time_jst))

# 処理にかかった時間を表示
print(paste("処理にかかった時間:", processing_time, "minutes"))


[1] "処理開始時刻（日本時間）: 2024-03-27 11:34:12.722806"
[1] "処理終了時刻（日本時間）: 2024-03-27 11:42:27.89077"
[1] "処理にかかった時間: 8.25279939572016 minutes"


In [ ]:
#@title (exactTest)
# # 処理開始時刻を記録
# start_time <- Sys.time()
# start_time_jst <- with_tz(start_time, tzone = "Asia/Tokyo")

# # 1連なのでBCV (square-rootdispersion)を0.1とする
# bcv <- 0.1

# # 全てのグループペアの組み合わせを作成
# group_combinations <- combn(levels(group), 2, simplify = FALSE)

# # 結果を格納するリスト
# results_list <- list()

# # 各グループペアに対してexactTestを実行
# for (pair in group_combinations) {
#  # 対象のグループのみを含むDGEListオブジェクトを作成
#  y_pair <- y[, group %in% pair]

#  # exactTestの実行
#  result <- exactTest(y_pair, dispersion=bcv^2)

#  # 結果の保存
#  results_list[[paste(pair, collapse = "-")]] <- topTags(result)
# }

# results_list


# result <- exactTest(y, dispersion=bcv^2)      #exact test
# topTags(result)


# # 処理終了時刻を記録
# end_time <- Sys.time()
# end_time_jst <- with_tz(end_time, tzone = "Asia/Tokyo")

# # 処理にかかった時間を計算
# processing_time <- end_time - start_time

# # 処理開始時刻と終了時刻を表示
# print(paste("処理開始時刻（日本時間）:", start_time_jst))
# print(paste("処理終了時刻（日本時間）:", end_time_jst))

# # 処理にかかった時間を表示
# print(paste("処理にかかった時間:", processing_time))

#プロット

In [ ]:
#@title  マンハッタンプロット

# 処理開始時刻を記録
start_time <- Sys.time()
start_time_jst <- with_tz(start_time, tzone = "Asia/Tokyo")


# FDR < 0.05 の遺伝子を赤色でプロット
# プロットのパラメータを設定
par(mfrow=c(4, 5))  # 4x5のグリッドにプロットを配置
# グループペアの組み合わせを取得
group_combinations <- combn(levels(group), 2, simplify = FALSE)

# 各グループペアに対してMAプロットを作成し、PDFファイルに保存
for (i in seq_along(group_combinations)) {
 pair <- group_combinations[[i]]

 # 対象のグループのみを含むDGEListオブジェクトを作成
 y_pair <- y[, group %in% pair]

 # exactTestの実行
 et_result <- exactTest(y_pair, dispersion=bcv^2)

 # FDR < 0.05 の遺伝子を赤色でプロット
 is.DEG <- et_result$table$FDR < 0.05
 DEG.names <- rownames(et_result$table)[is.DEG]

 # MAプロットのPDFファイル名を設定
 pdf_filename <- sprintf("MA_plot_%s_vs_%s.pdf", pair[1], pair[2])

 # PDFファイルを開く
 pdf(pdf_filename)

 # MAプロットを描画
 plotSmear(et_result, de.tags=DEG.names, cex=0.3, main=paste(pair[1], "vs", pair[2]))

 # デバイスを閉じる（PDFファイルを保存）
 dev.off()
}

# 処理終了時刻を記録
end_time <- Sys.time()
end_time_jst <- with_tz(end_time, tzone = "Asia/Tokyo")

# 処理にかかった時間を計算
processing_time <- end_time - start_time

# 処理開始時刻と終了時刻を表示
print(paste("処理開始時刻（日本時間）:", start_time_jst))
print(paste("処理終了時刻（日本時間）:", end_time_jst))

# 処理にかかった時間を表示
print(paste("処理にかかった時間:", processing_time, "minutes"))

[1] "処理開始時刻（日本時間）: 2024-03-23 09:12:22.188708"
[1] "処理終了時刻（日本時間）: 2024-03-23 09:16:47.706528"
[1] "処理にかかった時間: 4.42529699802399"


#遺伝子リスト

In [ ]:
# インストールパッケージ
# install.packages("openxlsx")

# ロードパッケージ
# library(readxl)
# library(openxlsx)
# library(tidyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr   1.1.4     ✔ readr   2.1.5
✔ forcats 1.0.0     ✔ tibble  3.2.1
✔ ggplot2 3.4.4     ✔ tidyr   1.3.1
✔ purrr   1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract()   masks magrittr::extract()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::lag()       masks stats::lag()
✖ purrr::set_names() masks magrittr::set_names()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
# データの再読み込み
# 処理開始時刻を記録
start_time <- Sys.time()
start_time_jst <- with_tz(start_time, tzone = "Asia/Tokyo")

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/R"  #@param ["/content/drive/MyDrive/RNAseq/R", "option1", "option2"] {allow-input: true}
FILE <- "edgeR_result_bcv_0.1.txt"  #@param ["edgeR_result_bcv_0.1.txt", "/content/drive/MyDrive/RNAseq/R/edgeR_result_bcv_0.1.txt", "option2"] {allow-input: true}

# ワーキングディレクトリへ移動
setwd(DIR_WORK)
print(paste("現在のワーキングディレクトリ:", getwd()))
# list.files()

read_data <- read.table(FILE, stringsAsFactors=F, header=T) %>%
        mutate(Regulation = ifelse(logFC < 0, "down", "up"))

# read_data %>% colnames()
# read_data %>% distinct(Comparison)

# 処理終了時刻を記録
end_time <- Sys.time()
end_time_jst <- with_tz(end_time, tzone = "Asia/Tokyo")

# 処理にかかった時間を計算
processing_time <- end_time - start_time

# 処理開始時刻と終了時刻を表示
print(paste("処理開始時刻（日本時間）:", start_time_jst))
print(paste("処理終了時刻（日本時間）:", end_time_jst))

# 処理にかかった時間を表示
print(paste("処理にかかった時間:", processing_time, "seconds"))

[1] "現在のワーキングディレクトリ: /content/drive/MyDrive/RNAseq/R"
[1] "処理開始時刻（日本時間）: 2024-03-23 09:18:20.964164"
[1] "処理終了時刻（日本時間）: 2024-03-23 09:19:02.91261"
[1] "処理にかかった時間: 41.9484462738037"


In [ ]:
## サンプルIDのリスト
sample_ids <- c("PC10", "PC30", "PV", "SC10", "SC30", "SV")
tissues <- c("PFC", "HIP", "NAc")

## 遺伝子リストの書き出し
extract_gene_list(read_data, tissues, sample_ids)

#その他

In [ ]:
#@title  (遺伝子リスト作成)
# PFCの遺伝子リスト
data_PFC <- read_data %>%
        filter(Comparison == "PFC_PC10-PFC_PC30" |
                       Comparison == "PFC_PC10-PFC_PV"|
                       Comparison == "PFC_PC10-PFC_SC10"|
                       Comparison == "PFC_PC10-PFC_SC30"|
                       Comparison == "PFC_PC10-PFC_SV"|
                       Comparison == "PFC_PC30-PFC_PV"|
                       Comparison == "PFC_PC30-PFC_SC10"|
                       Comparison == "PFC_PC30-PFC_SC30"|
                       Comparison == "PFC_PC30-PFC_SV"|
                       Comparison == "PFC_PV-PFC_SC10"|
                       Comparison == "PFC_PV-PFC_SC30"|
                       Comparison == "PFC_PV-PFC_SV"|
                       Comparison == "PFC_SC10-PFC_SC30"|
                       Comparison == "PFC_SC10-PFC_SV"|
                       Comparison == "PFC_SC30-PFC_SV")


# HIPの遺伝子リスト
data_HIP <- read_data %>%
        filter(Comparison == "HIP_PC10-HIP_PC30" |
                       Comparison == "HIP_PC10-HIP_PV"|
                       Comparison == "HIP_PC10-HIP_SC10"|
                       Comparison == "HIP_PC10-HIP_SC30"|
                       Comparison == "HIP_PC10-HIP_SV"|
                       Comparison == "HIP_PC30-HIP_PV"|
                       Comparison == "HIP_PC30-HIP_SC10"|
                       Comparison == "HIP_PC30-HIP_SC30"|
                       Comparison == "HIP_PC30-HIP_SV"|
                       Comparison == "HIP_PV-HIP_SC10"|
                       Comparison == "HIP_PV-HIP_SC30"|
                       Comparison == "HIP_PV-HIP_SV"|
                       Comparison == "HIP_SC10-HIP_SC30"|
                       Comparison == "HIP_SC10-HIP_SV"|
                       Comparison == "HIP_SC30-HIP_SV")


# NAcの遺伝子リスト
data_NAc <- read_data %>%
        filter(Comparison == "NAc_PC10-NAc_PC30" |
                       Comparison == "NAc_PC10-NAc_PV"|
                       Comparison == "NAc_PC10-NAc_SC10"|
                       Comparison == "NAc_PC10-NAc_SC30"|
                       Comparison == "NAc_PC10-NAc_SV"|
                       Comparison == "NAc_PC30-NAc_PV"|
                       Comparison == "NAc_PC30-NAc_SC10"|
                       Comparison == "NAc_PC30-NAc_SC30"|
                       Comparison == "NAc_PC30-NAc_SV"|
                       Comparison == "NAc_PV-NAc_SC10"|
                       Comparison == "NAc_PV-NAc_SC30"|
                       Comparison == "NAc_PV-NAc_SV"|
                       Comparison == "NAc_SC10-NAc_SC30"|
                       Comparison == "NAc_SC10-NAc_SV"|
                       Comparison == "NAc_SC30-NAc_SV")


# PFCの遺伝子リスト
data_PFC_p <- data_PFC %>% filter(PValue < 0.05)
data_PFC_fdr <- data_PFC %>% filter(FDR < 0.05)

# data_PFC_fdr %>% distinct(Comparison)
data_PFC_fdr_PV_SV <- data_PFC_fdr %>% filter(Comparison == "PFC_PV-PFC_SV")
data_PFC_fdr_PV_SV_up <- data_PFC_fdr_PV_SV %>% filter(Regulation == "up")
data_PFC_fdr_PV_SV_down <- data_PFC_fdr_PV_SV %>% filter(Regulation == "down")

data_PFC_fdr_PC10_PV <- data_PFC_fdr %>% filter(Comparison == "PFC_PC10-PFC_PV")
data_PFC_fdr_PC10_PV_up <- data_PFC_fdr_PC10_PV %>% filter(Regulation == "up")
data_PFC_fdr_PC10_PV_down <- data_PFC_fdr_PC10_PV %>% filter(Regulation == "down")

data_PFC_fdr_PC30_PV <- data_PFC_fdr %>% filter(Comparison == "PFC_PC30-PFC_PV")
data_PFC_fdr_PC30_PV_up <- data_PFC_fdr_PC30_PV %>% filter(Regulation == "up")
data_PFC_fdr_PC30_PV_down <- data_PFC_fdr_PC30_PV %>% filter(Regulation == "down")


# NAcの遺伝子リスト
data_NAc_p <- data_NAc %>% filter(PValue < 0.05)
data_NAc_fdr <- data_NAc %>% filter(FDR < 0.05)

# data_NAc_fdr %>% distinct(Comparison)
data_NAc_fdr_PV_SV <- data_NAc_fdr %>% filter(Comparison == "NAc_PV-NAc_SV")
data_NAc_fdr_PV_SV_up <- data_NAc_fdr_PV_SV %>% filter(Regulation == "up")
data_NAc_fdr_PV_SV_down <- data_NAc_fdr_PV_SV %>% filter(Regulation == "down")

data_NAc_fdr_PC10_PV <- data_NAc_fdr %>% filter(Comparison == "NAc_PC10-NAc_PV")
data_NAc_fdr_PC10_PV_up <- data_NAc_fdr_PC10_PV %>% filter(Regulation == "up")
data_NAc_fdr_PC10_PV_down <- data_NAc_fdr_PC10_PV %>% filter(Regulation == "down")

data_NAc_fdr_PC30_PV <- data_NAc_fdr %>% filter(Comparison == "NAc_PC30-NAc_PV")
data_NAc_fdr_PC30_PV_up <- data_NAc_fdr_PC30_PV %>% filter(Regulation == "up")
data_NAc_fdr_PC30_PV_down <- data_NAc_fdr_PC30_PV %>% filter(Regulation == "down")


# HIPの遺伝子リスト
data_HIP_p <- data_HIP %>% filter(PValue < 0.05)
data_HIP_fdr <- data_HIP %>% filter(FDR < 0.05)

# data_HIP_fdr %>% distinct(Comparison)
data_HIP_fdr_PV_SV <- data_HIP_fdr %>% filter(Comparison == "HIP_PV-HIP_SV")
data_HIP_fdr_PV_SV_up <- data_HIP_fdr_PV_SV %>% filter(Regulation == "up")
data_HIP_fdr_PV_SV_down <- data_HIP_fdr_PV_SV %>% filter(Regulation == "down")

data_HIP_fdr_PC10_PV <- data_HIP_fdr %>% filter(Comparison == "HIP_PC10-HIP_PV")
data_HIP_fdr_PC10_PV_up <- data_HIP_fdr_PC10_PV %>% filter(Regulation == "up")
data_HIP_fdr_PC10_PV_down <- data_HIP_fdr_PC10_PV %>% filter(Regulation == "down")

data_HIP_fdr_PC30_PV <- data_HIP_fdr %>% filter(Comparison == "HIP_PC30-HIP_PV")
data_HIP_fdr_PC30_PV_up <- data_HIP_fdr_PC30_PV %>% filter(Regulation == "up")
data_HIP_fdr_PC30_PV_down <- data_HIP_fdr_PC30_PV %>% filter(Regulation == "down")

In [ ]:
#@title  (遺伝子リスト書き出し)
# ファイルパス
OUT_FILE_1 <- "PFC_fdr.xlsx"  #@param ["PFC_fdr.xlsx", "/content/drive/MyDrive/RNAseq/R/PFC_fdr.xlsx", "option2"] {allow-input: true}
OUT_FILE_2 <- "NAc_fdr.xlsx"  #@param ["NAc_fdr.xlsx", "/content/drive/MyDrive/RNAseq/R/NAc_fdr.xlsx", "option2"] {allow-input: true}
OUT_FILE_3 <- "HIP_fdr.xlsx"  #@param ["HIP_fdr.xlsx", "/content/drive/MyDrive/RNAseq/R/HIP_fdr.xlsx", "option2"] {allow-input: true}

# PFCの遺伝子リスト
out_list <- list("PV_SV" = data_PFC_fdr_PV_SV
            , "PV_SV_up" = data_PFC_fdr_PV_SV_up
            , "PV_SV_down" = data_PFC_fdr_PV_SV_down
            , "PC10_PV" = data_PFC_fdr_PC10_PV
            , "PC10_PV_up" = data_PFC_fdr_PC10_PV_up
            , "PC10_PV_down" = data_PFC_fdr_PC10_PV_down
            , "PC30_PV" = data_PFC_fdr_PC30_PV
            , "PC30_PV_up" = data_PFC_fdr_PC30_PV_up
            , "PC30_PV_down" = data_PFC_fdr_PC30_PV_down
)
write.xlsx(out_list, OUT_FILE_1, rowNames = F)

# NAcの遺伝子リスト
out_list <- list("PV_SV" = data_NAc_fdr_PV_SV
                 , "PV_SV_up" = data_NAc_fdr_PV_SV_up
                 , "PV_SV_down" = data_NAc_fdr_PV_SV_down
                 , "PC10_PV" = data_NAc_fdr_PC10_PV
                 , "PC10_PV_up" = data_NAc_fdr_PC10_PV_up
                 , "PC10_PV_down" = data_NAc_fdr_PC10_PV_down
                 , "PC30_PV" = data_NAc_fdr_PC30_PV
                 , "PC30_PV_up" = data_NAc_fdr_PC30_PV_up
                 , "PC30_PV_down" = data_NAc_fdr_PC30_PV_down
)
write.xlsx(out_list, OUT_FILE_2, rowNames = F)

# HIPの遺伝子リスト
out_list <- list("PV_SV" = data_HIP_fdr_PV_SV
                 , "PV_SV_up" = data_HIP_fdr_PV_SV_up
                 , "PV_SV_down" = data_HIP_fdr_PV_SV_down
                 , "PC10_PV" = data_HIP_fdr_PC10_PV
                 , "PC10_PV_up" = data_HIP_fdr_PC10_PV_up
                 , "PC10_PV_down" = data_HIP_fdr_PC10_PV_down
                 , "PC30_PV" = data_HIP_fdr_PC30_PV
                 , "PC30_PV_up" = data_HIP_fdr_PC30_PV_up
                 , "PC30_PV_down" = data_HIP_fdr_PC30_PV_down
)
write.xlsx(out_list, OUT_FILE_3, rowNames = F)

#info

【R言語】openxlsxを使ったエクセルファイルの入力・出力  
https://multivariate-statistics.com/2021/12/07/r-programming-xlsx-files/

In [ ]:
# ロードパッケージ
# library(readxl)
# library(openxlsx)
# library(tidyverse)

In [ ]:
# データの再読み込み
# 処理開始時刻を記録
start_time <- Sys.time()
start_time_jst <- with_tz(start_time, tzone = "Asia/Tokyo")

# ワーキングディレクトリとファイルパス
DIR_WORK = "/content/drive/MyDrive/RNAseq/R"  #@param ["/content/drive/MyDrive/RNAseq/R", "option1", "option2"] {allow-input: true}
FILE_1 <- "PFC_fdr.xlsx"  #@param ["PFC_fdr.xlsx", "/content/drive/MyDrive/RNAseq/R/PFC_fdr.xlsx", "option2"] {allow-input: true}
FILE_2 <- "NAc_fdr.xlsx"  #@param ["NAc_fdr.xlsx", "/content/drive/MyDrive/RNAseq/R/NAc_fdr.xlsx", "option2"] {allow-input: true}
FILE_3 <- "HIP_fdr.xlsx"  #@param ["HIP_fdr.xlsx", "/content/drive/MyDrive/RNAseq/R/HIP_fdr.xlsx", "option2"] {allow-input: true}


# ワーキングディレクトリへ移動
setwd(DIR_WORK)
print(paste("現在のワーキングディレクトリ:", getwd()))
# list.files()

data_1 <- read.xlsx(FILE_1)
data_2 <- read.xlsx(FILE_2)
data_3 <- read.xlsx(FILE_3)

# read_data %>% colnames()
# read_data %>% distinct(Comparison)

# 処理終了時刻を記録
end_time <- Sys.time()
end_time_jst <- with_tz(end_time, tzone = "Asia/Tokyo")

# 処理にかかった時間を計算
processing_time <- end_time - start_time

# 処理開始時刻と終了時刻を表示
print(paste("処理開始時刻（日本時間）:", start_time_jst))
print(paste("処理終了時刻（日本時間）:", end_time_jst))

# 処理にかかった時間を表示
print(paste("処理にかかった時間:", processing_time))

[1] "現在のワーキングディレクトリ: /content/drive/MyDrive/RNAseq/R"
[1] "処理開始時刻（日本時間）: 2024-03-23 09:55:01.528148"
[1] "処理終了時刻（日本時間）: 2024-03-23 09:55:01.63698"
[1] "処理にかかった時間: 0.108831882476807"


In [ ]:
head(data_1)
head(data_2)
head(data_3)

,Comparison,Gene,logFC,logCPM,PValue,FDR,Regulation
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,PFC_PV-PFC_SV,ATP8,-3.289671,7.67321826,2.601975e-47,4.132716e-43,down
2,PFC_PV-PFC_SV,Xlr3b,-3.128095,2.53633242,5.035083e-21,3.998611e-17,down
3,PFC_PV-PFC_SV,Lcn2,6.098088,0.67528603,2.327209e-17,1.232102e-13,up
4,PFC_PV-PFC_SV,Gm53253,4.943170,0.46525391,1.055719e-13,4.191997e-10,up
5,PFC_PV-PFC_SV,Gm14296,-5.426160,0.38477841,8.318485e-12,2.642450e-08,down
6,PFC_PV-PFC_SV,S100a9,8.399507,-0.05825212,2.796574e-11,7.402997e-08,up


,Comparison,Gene,logFC,logCPM,PValue,FDR,Regulation
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,NAc_PV-NAc_SV,Srp54b,4.963901,1.6724701,6.435240e-23,1.022109e-18,up
2,NAc_PV-NAc_SV,Lcn2,3.848527,1.4039764,2.880752e-16,2.287749e-12,up
3,NAc_PV-NAc_SV,Clca3a1,3.034910,1.6415624,6.447029e-14,3.413272e-10,up
4,NAc_PV-NAc_SV,Prg4,1.810852,3.4306725,4.514337e-12,1.792530e-08,up
5,NAc_PV-NAc_SV,Rho,2.742447,0.9280214,5.974344e-09,1.897810e-05,up
6,NAc_PV-NAc_SV,Xlr3b,-1.725738,2.5528497,8.001195e-09,2.118050e-05,down


,Comparison,Gene,logFC,logCPM,PValue,FDR,Regulation
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,HIP_PV-HIP_SV,Avp,3.759459,2.3616164,8.213518e-24,1.304553e-19,up
2,HIP_PV-HIP_SV,Lcn2,9.764560,1.3930700,3.268190e-23,2.595433e-19,up
3,HIP_PV-HIP_SV,Shox2,8.897719,0.5755339,1.690338e-15,8.949214e-12,up
4,HIP_PV-HIP_SV,Pard6b,-2.292965,2.8310554,3.064983e-15,1.217028e-11,down
5,HIP_PV-HIP_SV,Foxp2,2.918662,1.8025174,3.504948e-14,1.113382e-10,up
6,HIP_PV-HIP_SV,Gm6710,2.160596,2.7682731,1.839031e-13,4.868221e-10,up
